# Skills-Gap Analysis
- Given a selected Job Title and a linkedin profile
- Return the missing suggested-skills for a qualified candidate

## 0. Required Packages

In [1]:
import sys
import os
import zipfile
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import preprocess_string,strip_punctuation, STOPWORDS, strip_multiple_whitespaces, remove_stopwords, stem_text, STOPWORDS, strip_numeric, strip_tags
sys.path.insert(0,'/usr/lib/chromium-browser/chromedrive')
from selenium.webdriver import ChromeOptions
from selenium import webdriver
import bs4 as bs
import urllib.request
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
import re

## 1. Scrape LinkedIn Profile

In [2]:
# define LinkedIn login function
def login_linkedin(driver):
    """
    returns webdriver following login to LinkedIn
    
    """
    driver.get ("https://www.linkedin.com")
    driver.find_element_by_id("session_key").send_keys("keeganheilman@gmail.com")
    driver.find_element_by_id("session_password").send_keys("LIN@an0thEr$")
    driver.find_element_by_class_name("sign-in-form__submit-button").send_keys(Keys.RETURN)
    return driver

# define LinkedIn profile scraper
def scrape_linkedinprofile(linkedinprofile_URL):
    """
    returns LinkedIn profile page source
    
    """
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless') ## Enable to avoid launching Chrome instance in foreground
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver = login_linkedin(driver)
    driver.get(linkedinprofile_URL)
    # sleep for 4s for page to ensure load
    time.sleep(4)
    
    source = driver.page_source
    driver.quit()
    return source

linkedinprofile_URL = 'https://ca.linkedin.com/in/lanamois'
source = scrape_linkedinprofile(linkedinprofile_URL)
bsoup = bs.BeautifulSoup(source, 'lxml')

## 2. Choose Job Title

In [3]:
# Download Indeed dataset
!kaggle datasets download -p ~/Documents/UM-MADS/courses/697_capstone/ -d promptcloud/indeed-job-posting-dataset # api copied from kaggle

Traceback (most recent call last):
  File "/home/friend/.local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/friend/.local/lib/python3.8/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/friend/.local/lib/python3.8/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/friend/.kaggle. Or use the environment method.


In [4]:
# Import Indeed dataset as Pandas Dataframe
path = '/home/friend/Documents/UM-MADS/courses/697_capstone/'
filename = 'indeed-job-posting-dataset.zip'

with zipfile.ZipFile(f'{path}{filename}', 'r') as zip_ref:
    zip_ref.extractall(f'{path}')

dat_indeed = pd.read_csv(f"{path}home/sdf/marketing_sample_for_trulia_com-real_estate__20190901_20191031__30k_data.csv")
dat_indeed.head()

,Job Title,Job Description,Job Type,Categories,Location,City,State,Country,Zip Code,Address,...,Employer Phone,Employer Logo,Companydescription,Employer Location,Employer City,Employer State,Employer Country,Employer Zip Code,Uniq Id,Crawl Timestamp
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,Del Taco is an American quick service restaura...,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,511f9a53920f4641d701d51d3589349f,2019-08-24 09:13:18 +0000
1,Operations Support Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Atlanta, GA 30342",Atlanta,GA,United States,30342,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Based in Atlanta, FOCUS Brands Inc. is an inno...",NaN,NaN,NaN,United States,NaN,4955daf0a3facbe2acb6c429ba394e6d,2019-09-19 08:16:55 +0000
2,Senior Product Manager - Data,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Chicago, IL",Chicago,IL,United States,NaN,NaN,...,NaN,NaN,Vibes Corp. reputation was built and establish...,NaN,NaN,NaN,United States,NaN,a0e0d12df1571962b785f17f43ceae12,2019-09-18 02:13:10 +0000
3,Part-Time Office Concierge,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Festus, MO",Festus,MO,United States,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,56e411fd731f76ac916bf4fb169250e9,2019-10-24 16:39:13 +0000
4,Print & Marketing Associate,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Staples is The Worklife Fulfillment Company, h...","Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,3fff5c0ad6981bf4bff6260bd5feab63,2019-08-24 22:29:10 +0000


In [5]:
# Obtain Job Titles
job_titles = dat_indeed['Job Title']


# Selected Job Title == index

job_title = list(job_titles).index('Content & Digital Marketing Associate')
# job_title = list(job_titles).index('Enterprise Account Executive')
# job_title = list(job_titles).index('Education Community Manager')
dat_indeed.loc[job_title,'Job Title']
job_titles[4203]

'Content & Digital Marketing Associate'

## 3. Calculate Missing Suggested-Skills for Qualified Candidates

In [6]:
# Parse LinkedIn page source for relevant information
# Still working through this section, meanwhile will use the following for testing
profile_source = "Lana Moiseeva\n \n\n\n      UI/UX Designer at Aviva Canada\n    \n \n\n        Toronto, Ontario, Canada\n      \n\n\n            Contact info\n          \n\n\n\n\n\n\n\n\n\n\n    Aviva Canada\n  \n\n\n\n\n\n\n\n\n\n\n    BrainStation\n  \n\n\n\n\n\n\n\n\n\n\n500+ connections\n        \n\n\n\n\n\n\n\n\n\n\n    Connect\n\n\n\n \n\n\n\n    Message\n\n\n\nMore\n\n\n\n\n\n\n\n\nShare profile in a message\nShare Lana Moiseeva’s profile via message\n\n\n\n\n\n\n\nSave to PDF\nSave Lana Moiseeva’s profile to PDF\n\n\n\n\n\n\n\nFollow\nFollow Lana Moiseeva\n\n\n\n\n\n\n\nReport / Block\nReport or block Lana Moiseeva’s profile\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n \n\n  Highlights\n\n\n\n\n\n\n \n\n\n    About\n  \n\n\n\n    I am a UX/UI Designer with an educational psychology and graphic design background.\xa0I started my career in an advertising agency and created multiple designs for various brands. I am passionate about working with people and have 5+ years of experience in teaching Adobe Design Suite. As a UX designer I am inspired by applying a user-centered approach to solve real-world problems and adopting new technologies to make this process more efficient and joyful.\xa0⦁\tSoftware Expertise: Adobe Xd, Photoshop, Illustrator, Lightroom, Corel Draw, Figma.\xa0⦁\tUX Skills: research, user Interviews, user observation, surveys, sketching, user personas, information architecture, wireframes, user flows, task flows, usability testing, low-fidelity prototyping, high-fidelity prototyping.⦁\tUI Skills: UI prototyping, choosing colors, typography, application icons.⦁\tGraphic Design Skills: digital/print advertising products, brand identities, consumer packaging, illustrations, professional magazine photo retouching.\xa0Feel free to visit my portfolio by following link https://lanamoiseeva.comI am open for cooperation. Please do not hesitate to contact me at lanamoiseeva7@gmail.com.\n  \n      \n…\n\n          see more\n        \n\n\n\n\n  \n\n\n    Activity\n  \n\n \n\n\n      1,244 followers\n    \n\n\n\n\n \n\n\n\n \n\n\n \n\n\n\n\n    incredible\n  \n\n\n\n\n\n    Lana commented\n  \n\n\n \n\n\n\n\n\n\n \n    See all activity\n  \n\n    See all activity\n  \n\n\n\n\n\n \n\n\n    Experience\n  \n\n\n  \n\n \n\n\n\nUI/UX Designer\nCompany Name\n\n      Aviva Canada\n        Contract Full-time\n\n\n\nDates Employed\nMar 2021 – Present\n\n\nEmployment Duration\n5 mos\n\n\n\nLocation\nToronto, Ontario, Canada\n\n\n\n\n \n \n\n  \n\n \n\n\n\nMultidisciplinary [ Graphic, UX/UI ] Designer \nCompany Name\n\n      Upwork\n        Freelance\n\n\n\nDates Employed\n2020 – Present\n\n\nEmployment Duration\n1 yr\n\n\n\nLocation\nToronto, Ontario, Canada\n\n\n\n\n\n\n    ⦁  conducted UX research & user testing. ⦁  created sketches, wireframes, user personas, information architecture, task flows, low and high fidelity prototypes. ⦁  established style guides, selected typography, colors, application icons and the general look.\xa0\n  \n      \n…\n\n          see more\n        \n\n\n\n \n\n  \n\n \n\n\n\nAdobe Design Suite Instructor \nCompany Name\n\n      Superprof\n        Freelance\n\n\n\nDates Employed\n2017 – Present\n\n\nEmployment Duration\n4 yrs\n\n\n\nLocation\nToronto, Ontario, Canada\n\n\n\n\n\n\n    ⦁  taught software applications and graphic design with focus on unique needs of each student, which allowed them to reach their specific goals. ⦁  created graphic design & animation training courses for students.\n  \n\n\n \n\n  \n\n \n\n\n\nGraphic Designer\nCompany Name\n\n      GoodPoint Software\n \n\n\nDates Employed\nNov 2017 – Mar 2020\n\n\nEmployment Duration\n2 yrs 5 mos\n\n\n\n\n\n\n\n\n    Developed the company\'s name, logo, symbol, corporate style and its image in social media. Selected colors, typography and application icons. Created multiple interfaces for 35 Microsoft SharePoint Apps from official Microsoft Office Store.\n  \n\n\n \n\n  \n\n \n\n\n\nTeacher of Graphic Design Applications\nCompany Name\n\n      «Specialist» Computer Training Center\n        Permanent Full-time\n\n\n\nDates Employed\nApr 2011 – Oct 2013\n\n\nEmployment Duration\n2 yrs 7 mos\n\n\n\nLocation\nMoscow, Russia\n\n\n\n\n\n\n    One of the Top Instructors based on students’ evaluations at the leading Russian Training Center Specialist. Was actively involved in the development of 9 courses  and prepared students to get official certificates and diplomas.\n  \n\n\n \n\n \n\n\n\n\n\n    Education\n  \n\n\n\n\n \n\n\n\n\nBrainStation\n\nDegree Name\nCertificate\n\n\nField Of Study\nUX Design\n\n \n\nDates attended or expected graduation\n\n2020 – 2020\n\n\n\n\n\n  User Experience Design course shows best practices for defining problems to be solved, conducting UX research, interviewing users, developing personas, wireframes, information architecture and high/low-fidelity prototyping.\n\n\n\n Media (1)\nThis position has 1 media\n\n\n\n\n\n\n  \n\n\n \n\n\n \n\n830-781-697.pdf\n\n\n\n\n    830-781-697.pdf\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSholokhov Moscow State University for Humanities, Russia\n\nDegree Name\nBachelor’s degree, Educational Psychologist\n\n\nField Of Study\nPsychology\n\n \n\nDates attended or expected graduation\n\n2010\n\n\n\n\n \n\n\n\n\n\n\n\n    Licenses & certifications\n  \n\n\n\n\n \n\n\n\n\n    Certificate «Color Theory for Designers»\n  \n\n\n        Issuing authority\n      \n\n        Coursera\n      \n\n\n\n        Issued date and, if applicable, expiration date of the certification or license\n      \nIssued Jul 2020No Expiration Date\n\n\n\n\n\n\n\n          See credential\n        \n\n\n\n\n \n\n\n\n\n    Certificate «Graphic Design Theory for Beginners»\n  \n\n\n        Issuing authority\n      \n\n        Coursera\n      \n\n\n\n        Issued date and, if applicable, expiration date of the certification or license\n      \nIssued Jun 2020No Expiration Date\n\n\n\n\n\n\n\n          See credential\n        \n\n\n\n\n \n\n\n\n\n    Adobe Certified Expert in Photoshop CS5\n  \n\n\n        Issuing authority\n      \n\n        Adobe\n      \n\n\n\n        Issued date and, if applicable, expiration date of the certification or license\n      \nIssued Mar 2012No Expiration Date\n\n\n\n\n        Credential Identifier\n      \n\n        Credential ID ID #ADB288577\n      \n\n\n\n\n\n\n          See credential\n        \n\n\n\n\n\n\n\n\n    Volunteer experience\n  \n\n\n\n\n \n\n\n\n Energy Exchange Program Volunteer\n\nCompany Name\nCity Dance Corps\n\n\nDates volunteered\nJan 2018 – Sep 2019\n\n\nVolunteer duration\n1 yr 9 mos\n\n\nCause\n\nArts and Culture\n\n\n\n\n  Worked as a receptionist welcoming clients and signing them in, answered questions, maintained classrooms and conducted customer surveys.\n\n\n\n\n\n \n\n\n\nSupport Volunteer \n\nCompany Name\nThe Women\'s Centre of Calgary\n\n\nDates volunteered\n2015\n\n\nVolunteer duration\nless than a year\n\n\nCause\n\nSocial Services\n\n\n\n\n  Volunteered to help support women\'s mental health via sharing experiences and friendly conversations. In addition, processed administrative work orders.\n\n\n\n\n\n \n\n\n\nMeal Serve Volunteer\n\nCompany Name\nCalgary Drop In & Rehab Centre Society\n\n\nDates volunteered\n2015\n\n\nVolunteer duration\nless than a year\n\n\nCause\n\nSocial Services\n\n\n\n\n  Helped at homeless-serving sector to serve food to people staying at Rehab Center.\n\n\n\n\n\n\n\n\n\n \n\n\n    Skills & endorsements\n  \n\n\n\n\n\n\n\n\n\n \n            Adobe Creative Suite\n          \n\n \n See 1 endorsement for Adobe Creative Suite\n1\n\n\n\n\n\n\n\n\n Steve Cockrille has given an endorsement for this skill\n \n\n\n \n\n\n\n\n\n\n \n            Graphic Design\n          \n\n \n See 1 endorsement for Graphic Design\n1\n\n\n\n\n\n\n\n\n Steve Cockrille has given an endorsement for this skill\n \n\n\n \n\n\n\n\n\n\n\n          UX/UI Design\n        \n \n\n\n\n\n\n\n\n\n\n\n                      Show more\n                    \n\n                      Show all of Lana’s skills\n                    \n\n\n\n\n\n\n\n \n\nRecommendations\n\n\n      Received (0)\n      Given (0)\n\n\n\n\n        Lana hasn’t received any recommendations yet.\n      \n\n\n\n        Lana hasn’t given any recommendations yet.\n      \n\n\n\n\n\nAccomplishments\n\n \n\nLana has 15 courses\n15\n\n\nCourses\n \n\n\n\n\n\n\nAdobe Illustrator CC for Beginners\nAdobe Illustrator CC for Intermediate and Advanced users\nAdobe Lightroom Training Course\nAdobe Photoshop CC for Beginners\nAdobe Photoshop CC for Intermediate and Advanced users\nAutoCAD for Design\nColor Theory for Designers\nCorporate Identity: The Concepts of Branding\nCorporate Identity: The concepts of branding\nDigital Photo Processing\nGraphic Design Theory for Beginners\nGraphic Design in Adobe Photoshop CC\nIntroduction to User Experience Design\nProfessional Retouching\nUser Experience Design\n\n\n\n \n\nLana has 2 languages\n2\n\n\nLanguages\n \n\n\n\n\n\n\nEnglish\nRussian\n\n\n\n\n\n \nInterests\n\n \n\n\nThe Walt Disney Company\n\n\n4,454,937 followers\n\n\n \n\n\nNetflix\n\n\n7,331,042 followers\n\n\n \n\n\nAviva\n\n\n201,211 followers\n\n\n \n\n\nPro-X Exhibit\n\n\n170 followers\n\n\n \n\n\nAviva Canada\n\n\n61,407 followers\n\n\n \n\n\nCreative Circle\n\n\n425,120 followers\n\n\n\n See all\nSee all following\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n    English\n\n\n\n    Русский\n\n\n \n\n\n\n\n \n\n\n\n\n\n \n \n\n"
profile = pd.DataFrame({'resume': [bs.BeautifulSoup(profile_source, "lxml").text.replace('\n', ' ')]})

In [7]:
# Obtain job description of Selected job
job =  pd.DataFrame({'job_description': [bs.BeautifulSoup(dat_indeed.loc[job_title,'Job Description'], "lxml").text.replace('\n', ' ')]})

In [8]:
# Obtain skills set
skills_text_files = ['Skills.txt',
#                      'Technology Skills.txt',
                     'Knowledge.txt',
#                      'Education, Training, and Experience.txt',
#                      'Education, Training, and Experience Categories.txt'
                    ]
skills = pd.DataFrame()
for filename in skills_text_files:
    df_temp = pd.read_csv(f'{path}{filename}', delimiter = "\t")
    if ('Element Name' in df_temp.columns):
        df_temp['Commodity Title'] = df_temp['Element Name']
        df_temp.rename(columns={'Element Name':'Example'}, inplace = True)
    print(df_temp.columns)
    df_temp = df_temp[['Example', 'Commodity Title']]
    skills = pd.concat([skills, df_temp])
skills

Index(['O*NET-SOC Code', 'Element ID', 'Example', 'Scale ID', 'Data Value',
       'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound',
       'Recommend Suppress', 'Not Relevant', 'Date', 'Domain Source',
       'Commodity Title'],
      dtype='object')
Index(['O*NET-SOC Code', 'Element ID', 'Example', 'Scale ID', 'Data Value',
       'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound',
       'Recommend Suppress', 'Not Relevant', 'Date', 'Domain Source',
       'Commodity Title'],
      dtype='object')


,Example,Commodity Title
0,Reading Comprehension,Reading Comprehension
1,Reading Comprehension,Reading Comprehension
2,Active Listening,Active Listening
3,Active Listening,Active Listening
4,Writing,Writing
...,...,...
57613,Telecommunications,Telecommunications
57614,Communications and Media,Communications and Media
57615,Communications and Media,Communications and Media
57616,Transportation,Transportation


In [9]:
# Tokenize both LinkedIn profile and Job Description of Selected Job
def clean_tokenize_text(df, column):
    CUSTOM_STOP_WORDS = ['â¢', 'â', 'â', 'â¢', 'â','®', '�',
                         '\\u200','comment','connect','message','\\xa0',
                         'share','mo','month','follow','report',
                         'block','profile','yr','year','date',
                         'contact','jan','january','feb','february',
                         'mar','march','apr','april','may','jun',
                         'june','jul','july','aug','august','sep',
                         'sept','september','oct','october','nov',
                         'november','dec','december','yddscsgylniquit',
                         'srmysiridmcwdydzygf'
                        ]
    CUSTOM_STEM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_numeric, stem_text]
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_numeric]
    tokens = []
    s_tokens = []
    stem_tokens = []
    s_stem_tokens = []
    for s in df[column]:
        tokens = preprocess_string(s, CUSTOM_FILTERS)
        tokens = [x for x in tokens if len(x) > 1]
        
        stem_tokens = preprocess_string(s, CUSTOM_STEM_FILTERS)
        stem_tokens = [x for x in stem_tokens if len(x) > 1]
        
        s_stem_tokens.append(stem_tokens)
        s_tokens.append(tokens)
    df['tokens'] = s_tokens
    df['stem_tokens'] = s_stem_tokens
    return df

# Extract tokens of the LinkedIn profile, the job description, and the skills list
profile = clean_tokenize_text(profile.copy(), 'resume')
job = clean_tokenize_text(job.copy(), 'job_description')
skills = clean_tokenize_text(skills.copy(), 'Example')
# skills = skills[skills['stem_tokens'].str.len() < 2] ## To eliminate bigrams+ from skills stems

profile_stem_tokens = set(profile.loc[0,'stem_tokens'])
job_stem_tokens = set(job.loc[0,'stem_tokens'])
skills_exp = skills.explode('stem_tokens')
skills_stem_tokens = list(skills_exp['stem_tokens'].unique())


In [10]:
# Identify skills (skills_stem_tokens) from the job description (job_stem_tokens) NOT found in the user's LinkedIn profile (profile_stem_tokens)

job_skills = []
missing_skills = None

for j_stem_token in job_stem_tokens:
    
    if j_stem_token not in profile_stem_tokens:
        # job description token NOT FOUND in linkedin profile.
        
        if j_stem_token in skills_stem_tokens: 
            # job desription token IS FOUND in general skills.
            
            # append job description token to the list of skills found in the job description,
            #   but not found in the linked in profile.
            job_skills.append(j_stem_token)
                      
# create mask for skills DataFrame where Truth=SKILL_NOT_FOUND_IN_LINKEDIN_PROFILE, False=SKILL_NOT_IN_JOB_DESCRIPTION_OR_SKILL_FOUND_IN_LINKEDIN_PROFILE
skills_mask = skills_exp['stem_tokens'].isin(job_skills)

## 4. Return Missing Skills

In [11]:
missing_skills = list(skills_exp.loc[skills_mask, 'Commodity Title'].explode().unique())
missing_skills

['Critical Thinking',
 'Active Learning',
 'Learning Strategies',
 'Monitoring',
 'Coordination',
 'Operation Monitoring',
 'Equipment Maintenance',
 'Troubleshooting',
 'Time Management',
 'Management of Financial Resources',
 'Management of Material Resources',
 'Management of Personnel Resources',
 'Administration and Management',
 'Sales and Marketing',
 'Communications and Media']